# Exploration of Merging Data Files

This notebook was built to explore how to best merge the Cisco Umbrella 1m Domains file and the Bambenek Consulting DGA High-Confidence Feed.

In [1]:
# Libraries
import zipfile
import numpy as np
import pandas as pd
import re
import os
import time
import sys

# pandas config
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)


In [2]:
# os.getcwd()
os.chdir('../data/2018-10-06/')

In [3]:
# open ZIP files in cwd and extract all the files and store them to disk
zip_file = zipfile.ZipFile('top-1m.csv.zip', 'r')
zip_file.extractall('./')


In [4]:
# importer for Cisco Umbrella 1m Domains
MB = 1024*1024

start_time = time.time()
df_cisco = pd.read_csv('top-1m.csv', sep=',', skip_blank_lines=True, usecols=[1], names=['domain'], engine='c')
end_time = time.time()
read_time = end_time - start_time

# add columns
df_cisco['malware'] = 'NA'
df_cisco['date'] = np.datetime64('2018-10-06')
df_cisco['dga'] = 0

# analyze
print('Time to import Cisco Umbrella: {:>5.3f}s'.format(read_time))
print('Size of dataset in memory: {:>5.3f}MB'.format(sys.getsizeof(df_cisco)/MB))
print('Top 20 rows:')
print(df_cisco.head(n=20))
print('Summary stats:')
print(df_cisco.describe(include='all'))



Time to import Cisco Umbrella: 1.081s
Size of dataset in memory: 146.135MB
Top 20 rows:
                         domain malware       date  dga
0                   netflix.com      NA 2018-10-06    0
1        api-global.netflix.com      NA 2018-10-06    0
2              prod.netflix.com      NA 2018-10-06    0
3         push.prod.netflix.com      NA 2018-10-06    0
4                    google.com      NA 2018-10-06    0
5                www.google.com      NA 2018-10-06    0
6                 microsoft.com      NA 2018-10-06    0
7               doubleclick.net      NA 2018-10-06    0
8                  facebook.com      NA 2018-10-06    0
9           ichnaea.netflix.com      NA 2018-10-06    0
10            g.doubleclick.net      NA 2018-10-06    0
11  safebrowsing.googleapis.com      NA 2018-10-06    0
12  googleads.g.doubleclick.net      NA 2018-10-06    0
13          clients4.google.com      NA 2018-10-06    0
14         google-analytics.com      NA 2018-10-06    0
15           dat

In [5]:
# importer for Bambenek Consulting DGA High-Confidence Feed
start_time = time.time()
df_dga = pd.read_csv('dga-feed-high.csv', sep=',', header=16, skip_blank_lines=True, usecols=[0,1,2], names=['domain', 'malware', 'date'], dtype={0:str,1:str}, parse_dates=[2], engine='c')

# modify the malware column
reg = re.compile(r'(^.*? )')
prefix_len = len('Domain used by ')
df_dga['malware'] = df_dga['malware'].str[prefix_len:]
df_dga['malware'] = df_dga['malware'].str.extract(r'(^.*? )', expand=True)
df_dga['malware'] = df_dga['malware'].str.lower()
df_dga['malware'] = df_dga['malware'].str.strip()

# add columns
df_dga['dga'] = 1

end_time = time.time()
read_time = end_time - start_time

print('Time to import Bambeneck DGA Feed: {:>5.3f}s'.format(read_time))
print('Size of dataset in memory: {:>5.3f}MB'.format(sys.getsizeof(df_dga)/MB))
print('Top 20 rows:')
print(df_dga.head(n=20))
print('Summary stats:')
print(df_dga.describe(include='all'))


Time to import Bambeneck DGA Feed: 2.642s
Size of dataset in memory: 55.453MB
Top 20 rows:
                   domain       malware       date  dga
0     lepqlimlanmrnkm.biz  cryptolocker 2018-10-04    1
1      ycubddqnrscxnat.ru  cryptolocker 2018-10-04    1
2     mvfnryumyqqueim.org  cryptolocker 2018-10-04    1
3   atkxjtyoqvgbnkl.co.uk  cryptolocker 2018-10-04    1
4    vavbdfduaykdkbt.info  cryptolocker 2018-10-04    1
5     wmbgjajcrpxrcyw.com  cryptolocker 2018-10-04    1
6     wrlxjvlvycogiuu.net  cryptolocker 2018-10-04    1
7     xeqdpqrdqscujcg.biz  cryptolocker 2018-10-04    1
8      tiaihxmvcjwfkqa.ru  cryptolocker 2018-10-04    1
9     uufnnssdtaktcvy.org  cryptolocker 2018-10-04    1
10  uapfnouwbmbibdr.co.uk  cryptolocker 2018-10-04    1
11   vmuktjbesdowcry.info  cryptolocker 2018-10-04    1
12    tnhupouqitvkqno.com  cryptolocker 2018-10-04    1
13    hlmfhgivbfbyalb.net  cryptolocker 2018-10-04    1
14    viwsefdejdwsxno.biz  cryptolocker 2018-10-04    1
15     jgcdvw

In [6]:
df_dga.dtypes

domain             object
malware            object
date       datetime64[ns]
dga                 int64
dtype: object

In [7]:
# Sort malware by number of domains in the dataset
malware_counts = pd.DataFrame(df_dga.groupby('malware')['domain'].nunique())
malware_counts.sort_values(by=['domain'], ascending=False)

,domain
malware,
tinba,66688
post,66000
ramnit,56174
necurs,32768
murofet,28520
qakbot,20000
pykspa,14215
ranbyus,13960
shiotob/urlzone/bebloh,12521


In [8]:
# merge both datasets and write to disk
df_merged = df_cisco.append(df_dga, ignore_index=True)
df_merged

,domain,malware,date,dga
0,netflix.com,NA,2018-10-06,0
1,api-global.netflix.com,NA,2018-10-06,0
2,prod.netflix.com,NA,2018-10-06,0
3,push.prod.netflix.com,NA,2018-10-06,0
4,google.com,NA,2018-10-06,0
5,www.google.com,NA,2018-10-06,0
6,microsoft.com,NA,2018-10-06,0
7,doubleclick.net,NA,2018-10-06,0
8,facebook.com,NA,2018-10-06,0
9,ichnaea.netflix.com,NA,2018-10-06,0


In [9]:
# write merged dataset to a CSV file
df_merged.to_csv(path_or_buf='merged.csv', sep=',', header=True, index=False)